In [1]:
%matplotlib widget
import numpy as np
import matplotlib.pyplot as plt
import xarray as xr
import gsw
from oceans import cor_beta

In [2]:
bathy_nc = '/Users/tomconnolly/work/Data/bathy/CRM/etopo1_bedrock_westcoast.nc'

In [3]:
b = xr.open_dataset(bathy_nc)

In [4]:
b

<xarray.Dataset>
Dimensions:  (lat: 1261, lon: 1201)
Coordinates:
  * lat      (lat) float64 30.0 30.02 30.03 30.05 ... 50.95 50.97 50.98 51.0
  * lon      (lon) float64 -135.0 -135.0 -135.0 -134.9 ... -115.0 -115.0 -115.0
Data variables:
    crs      |S1 ...
    Band1    (lat, lon) float32 ...
Attributes:
    GDAL_TIFFTAG_RESOLUTIONUNIT:  2
    GDAL_TIFFTAG_XRESOLUTION:     72.0
    GDAL_TIFFTAG_YRESOLUTION:     72.0
    Conventions:                  CF-1.5
    GDAL:                         GDAL 2.1.2, released 2016/10/24
    history:                      Tue Aug 08 14:46:53 2017: GDAL CreateCopy( ...

In [5]:
mlon = -123
#mlat = 34+50/60
mlat = 35+8.5/60

In [6]:
b['h'] = b.Band1

In [8]:
ii = int(np.argmin(np.abs(b.lat-mlat)))
jj = int(np.argmin(np.abs(b.lon-mlon)))

In [9]:
plt.figure()
plt.contour(b.lon,b.lat,b.h,np.arange(-4500,-3000,50))
plt.plot(mlon,mlat,'ro')
si = 8
plt.plot(b.lon[jj-si],b.lat[ii+si],'rx')
plt.plot(b.lon[jj+si],b.lat[ii-si],'rx')
plt.xlim([-124,-122])
plt.ylim([33,36])

FigureCanvasNbAgg()

(33, 36)

In [10]:
b.h[ii,jj]

<xarray.DataArray 'h' ()>
array(-3962., dtype=float32)
Coordinates:
    lat      float64 35.13
    lon      float64 -123.0
Attributes:
    long_name:     GDAL Band Number 1
    grid_mapping:  crs

In [28]:
sepi = 3
dhdx = -(b.h[ii,jj+sepi]-b.h[ii,jj-sepi])/(111000*(b.lon[jj+sepi]-b.lon[jj-sepi])*np.cos(mlat*np.pi/180))
dhdy = -(b.h[ii+sepi,jj]-b.h[ii-sepi,jj])/(111000*(b.lat[ii+sepi]-b.lat[ii-sepi]))

In [29]:
b.h[ii+sepi,jj]

<xarray.DataArray 'h' ()>
array(-3960., dtype=float32)
Coordinates:
    lat      float64 35.18
    lon      float64 -123.0
Attributes:
    long_name:     GDAL Band Number 1
    grid_mapping:  crs

In [30]:
print(np.array(dhdx))
print(np.array(dhdy))
print(np.array(np.sqrt(dhdx**2+dhdy**2)))
np.array(b.lat[ii+sepi]-b.lat[ii-sepi])

-0.0005508538383998974
-0.0025225225225224863
0.002581968208153057


array(0.1)

In [31]:
b.lat

<xarray.DataArray 'lat' (lat: 1261)>
array([30.      , 30.016667, 30.033333, ..., 50.966667, 50.983333, 51.      ])
Coordinates:
  * lat      (lat) float64 30.0 30.02 30.03 30.05 ... 50.95 50.97 50.98 51.0
Attributes:
    standard_name:  latitude
    long_name:      latitude
    units:          degrees_north

In [32]:
f = gsw.f(mlat)
beta = np.array(cor_beta(mlat))
H = np.abs(b.h[ii,jj])

In [33]:
np.array(beta)

array(1.87191296e-11)

In [34]:
np.array(-dhdy*f/H)

array(5.34471963e-11)

In [35]:
beta_eff = H*np.array(np.sqrt(((beta/H-dhdy*f/H**2)**2 + (-dhdx*f/H**2)**2)))

In [36]:
beta_eff

<xarray.DataArray 'h' ()>
array(7.310405e-11)
Coordinates:
    lat      float64 35.13
    lon      float64 -123.0

In [37]:
(111000*(b.lat[ii+6]-b.lat[ii-6]))

<xarray.DataArray 'lat' ()>
array(22200.)

In [38]:
(b.lat[ii+8]-b.lat[ii-8])

<xarray.DataArray 'lat' ()>
array(0.266667)

In [40]:
for sepi in range(1,33):
    dhdx = -(b.h[ii,jj+sepi]-b.h[ii,jj-sepi])/(111000*(b.lon[jj+sepi]-b.lon[jj-sepi])*np.cos(mlat*np.pi/180))
    dhdy = -(b.h[ii+sepi,jj]-b.h[ii-sepi,jj])/(111000*(b.lat[ii+sepi]-b.lat[ii-sepi]))
    #print('dhdx',np.array(dhdx))
    #print('dhdy',np.array(dhdy))
    #print(np.array(np.sqrt(dhdx**2+dhdy**2)))
    print(np.array(b.lat[ii+sepi]-b.lat[ii-sepi]))
    print(np.array(H*np.array(np.sqrt(((beta/H-dhdy*f/H**2)**2 + (-dhdx*f/H**2)**2)))))

0.03333333333333144
1.5756166753289318e-10
0.06666666666666288
1.0799174393642469e-10
0.10000000000000142
7.310405082010148e-11
0.13333333333333286
1.0231739493795169e-10
0.1666666666666714
1.3173297040643642e-10
0.20000000000000284
1.6769635105403355e-10
0.23333333333333428
1.720503871106645e-10
0.2666666666666657
1.6442296141047042e-10
0.29999999999999716
1.1677048488685257e-10
0.3333333333333286
9.144817451993366e-11
0.36666666666666714
8.743927103295816e-11
0.3999999999999986
1.1750245050897986e-10
0.4333333333333371
1.4074659482271918e-10
0.46666666666666856
1.659723629618875e-10
0.5
1.9439918965591515e-10
0.5333333333333314
2.0017761158641312e-10
0.5666666666666629
1.9928717048995526e-10
0.6000000000000014
1.9179255499615705e-10
0.6333333333333329
1.7695710228742745e-10
0.6666666666666714
1.5959681408759083e-10
0.7000000000000028
1.4830077255512042e-10
0.7333333333333343
1.4323061553491103e-10
0.7666666666666657
1.197259225306942e-10
0.7999999999999972
8.79784954169961e-11
0.8333